In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision import datasets, transforms
import numpy as np
import os
import random
from model_structure import get_preprocessing_transforms,BCNN, train_model, evaluate_model

ImportError: cannot import name 'train_model' from 'model_structure' (c:\Users\hskay\OneDrive\Documents\GitHub\ML_ZK\Experiment_3_RGB_ADV\model_structure.py)

In [7]:
seed = 301
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [8]:
BATCH_SIZE = 32
LEARNING_RATE = 0.001
NUM_EPOCHS = 50
INPUT_SIZE = 32
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [9]:
# Get transforms
train_transform, val_transform = get_preprocessing_transforms(INPUT_SIZE)

In [12]:
model_number = 1
# Set data set directory
train_dir = f'data4model_{model_number}/train/'
val_dir = f'data4model_{model_number}/test/'

In [13]:
train_dataset = datasets.ImageFolder(
        root=train_dir,
        transform=train_transform
    )
    
val_dataset = datasets.ImageFolder(
        root=val_dir,
        transform=val_transform
    )

In [15]:
# Create data loaders
train_loader = DataLoader(
    train_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True, 
    num_workers=4,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=False, 
    num_workers=4,
    pin_memory=True
)

In [16]:
model = BCNN(input_channels=3).to(DEVICE)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [ ]:
num_epochs = NUM_EPOCHS
best_val_acc = 0.0
    
for epoch in range(num_epochs):
    # Training phase
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, labels in train_loader:
        inputs = inputs.to(DEVICE)
        labels = labels.float().to(DEVICE)
        
        optimizer.zero_grad()
        outputs = model(inputs).squeeze()
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        predicted = (outputs > 0.5).float()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    train_acc = 100 * correct / total
    
    # Validation phase
    model.eval()
    val_correct = 0
    val_total = 0
    
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.to(DEVICE)
            labels = labels.float().to(DEVICE)
            
            outputs = model(inputs).squeeze()
            predicted = (outputs > 0.5).float()
            
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()
    
    val_acc = 100 * val_correct / val_total
    
    print(f'Epoch [{epoch+1}/{num_epochs}]')
    print(f'Training Loss: {running_loss/len(train_loader):.4f}')
    print(f'Training Accuracy: {train_acc:.2f}%')
    print(f'Validation Accuracy: {val_acc:.2f}%')
    
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), 'best_model.pth')